# Stark Map

In this notebook

In [1]:
import importlib
import starktools as st

import numpy as np
from numba import jit, prange, njit
import scipy.linalg

In [2]:
defects = {
        0 : [0.29665648771, 0.038296666, 0.0075131, -0.0045476],
        1 : [0.06836028379, -0.018629228, -0.01233275, -0.0079527],
        2 : [0.002891328825, -0.006357704, 0.0003367, 0.0008395],
        3 : [0.00044737927, -0.001739217, 0.00010478, 3.31e-05],
        4 : [0.00012714167, -0.000796484, -9.85e-06, -1.9e-05],
        5 : [4.8729846e-05, -0.0004332281, -8.1e-06, 0],
        6 : [2.3047609e-05, -0.0002610672, -4.04e-06, 0]
}

In [3]:
importlib.reload(st)
nmin = 50
nmax = 55
m0 = st.MatrixH0(nmin, nmax, defects)
mH = st.MatrixHs(nmin, nmax, defects)

In [4]:
m0.shape

(315, 315)

In [5]:
m0.nmax

55

In [6]:
#CODATA 2014, DOI: 10.1103/RevModPhys.88.035009
c = 299792458.0 ## speed of Heght in vacuum
h = 6.626070040e-34
hbar = 1.054571800e-34

Ry = 10973731.568508
e = 1.6021766208e-19
m_e = 9.10938356e-31
alpha = 7.2973525664e-3
amu = 1.660539040e-27 # Kg   (NIST) unified atomic mass unit 


a_0 = hbar/ (m_e * c * alpha)
mu_B = e * hbar / (2.0 * m_e)


# Helium
mHe = 4.0026*amu
mHeIon = mHe - m_e
reducedMass = (mHeIon * m_e)/(mHeIon+m_e)


a_He  = (a_0*m_e)/reducedMass

R_He = Ry * reducedMass/m_e
E_He = 2*h*c*R_He
F_He = E_He/(a_He*e)

In [7]:
def map_maker1(m0_array, mH_array, field):
    fsteps = field.shape[0]
    neig = m0_array.shape[0]
    
    vals = np.zeros((fsteps, neig))
    for i in range(fsteps):
        fau = field[i] / F_He
        val = (scipy.linalg.eigh(m0_array + mH_array*fau, eigvals_only=True) * E_He)/(h*c*100)
        for j in range(neig):
            vals[i,j] = val[j]
    return vals

In [9]:
field = np.linspace(0, 30, 400)# V /m
v1 = map_maker1(np.asarray(m0), np.asarray(mH), field)

In [15]:
vals = np.zeros((10, 20))

In [25]:
from numba import types
types.Array

numba.core.types.npytypes.Array

Numba Support

In [ ]:
from numba import int32, float32 

In [108]:
import numpy.lib.mixins
from numba import int32, float32
from numba.experimental import jitclass

spec = [             # a simple scalar field
    ('array', float32[:]),          # an array field
]
@jitclass(spec)
class DiagonalArray(numpy.lib.mixins.NDArrayOperatorsMixin):
   def __init__(self, N, value):
       self._N = N
       self._i = value
       self.array = self.__array__
   def __repr__(self):
       return f"{self.__class__.__name__}(N={self._N}, value={self._i})"
   def __array__(self, dtype=None):
       return self._i * np.eye(self._N, dtype=None)
   def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
       if method == '__call__':
           N = None
           scalars = []
           for input in inputs:
               if isinstance(input, Number):
                   scalars.append(input)
               elif isinstance(input, self.__class__):
                   scalars.append(input._i)
                   if N is not None:
                       if N != self._N:
                           raise TypeError("inconsistent sizes")
                   else:
                       N = self._N
               else:
                   return NotImplemented
           return self.__class__(N, ufunc(*scalars, **kwargs))
       else:
           return NotImplemented

TypeError: class members are not yet supported: __hash__

In [102]:
class DiagonalArrayType(types.Array):
    def __init__(self):
        super(DiagonalArrayType, self).__init__(name='DiagonalArray')

In [103]:
from numba.extending import typeof_impl
@typeof_impl.register(DiagonalArray)
def _typeof_DiagnolArray(val, c):
    return types.Array

In [104]:
@jit(nopython=True)
def func_eig(interval):
    return np.linalg.eigvals(interval)

In [105]:
arr = DiagonalArray(5, 1)
np.asarray(arr+arr)

array([[2., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0.],
       [0., 0., 2., 0., 0.],
       [0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 2.]])

In [106]:
func_eig(arr)

AttributeError: type object 'Array' has no attribute '_code'

In [5]:
from numbers import Number
class DiagonalArray:
 def __init__(self, N, value):
     self._N = N
     self._i = value
 def __repr__(self):
     return f"{self.__class__.__name__}(N={self._N}, value={self._i})"
 def __array__(self, dtype=None):
     return self._i * np.eye(self._N, dtype=dtype)
 def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
     if method == '__call__':
         N = None
         scalars = []
         for input in inputs:
             if isinstance(input, Number):
                 scalars.append(input)
             elif isinstance(input, self.__class__):
                 scalars.append(input._i)
                 if N is not None:
                     if N != self._N:
                         raise TypeError("inconsistent sizes")
                 else:
                     N = self._N
             else:
                 return NotImplemented
         return self.__class__(N, ufunc(*scalars, **kwargs))
     else:
         return NotImplemented

In [27]:
HANDLED_FUNCTIONS = {}
class DiagonalArray(numpy.lib.mixins.NDArrayOperatorsMixin):
  def __init__(self, N, value):
      self._N = N
      self._i = value
  def __repr__(self):
      return f"{self.__class__.__name__}(N={self._N}, value={self._i})"
  def __array__(self, dtype=None):
      return self._i * np.eye(self._N, dtype=dtype)
#   def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
#       if method == '__call__':
#           N = None
#           scalars = []
#           for input in inputs:
#               # In this case we accept only scalar numbers or DiagonalArrays.
#               if isinstance(input, Number):
#                   scalars.append(input)
#               elif isinstance(input, self.__class__):
#                   scalars.append(input._i)
#                   if N is not None:
#                       if N != self._N:
#                           raise TypeError("inconsistent sizes")
#                   else:
#                       N = self._N
#               else:
#                   return NotImplemented
#           return self.__class__(N, ufunc(*scalars, **kwargs))
#       else:
#           return NotImplemented
#   def __array_function__(self, func, types, args, kwargs):
#      if func not in HANDLED_FUNCTIONS:
#          return NotImplemented
#      # Note: this allows subclasses that don't override
#      # __array_function__ to handle DiagonalArray objects.
#      if not all(issubclass(t, self.__class__) for t in types):
#          return NotImplemented
#      return HANDLED_FUNCTIONS[func](*args, **kwargs)

# def implements(np_function):
#    def decorator(func):
#        HANDLED_FUNCTIONS[np_function] = func
#        return func
#    return decorator

In [28]:
a = DiagonalArray(5,1)

In [29]:
a + a

array([[2., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0.],
       [0., 0., 2., 0., 0.],
       [0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 2.]])

In [30]:
a.shape

AttributeError: 'DiagonalArray' object has no attribute 'shape'